![Podmanlogo](Pictures/podman-logo.png)

# Running containers at system start

There are many situations in which you may want to start a container automatically when the system starts. A good example of this would be an edge environment, think of a small ruggered computer in a factory that runs containerized software using Podman. If there is a power failure the system will of course shut down, once the power is back the system will boot on and most likely you would like your containers to start automatically after boot. We can easily achieve this by taking advantage of systemd and Podman.

In most linux distributions systemd is the parent of all processes on the system, it is executed by the kernel and is responsible for starting all other processes. If we want to automatically start a process, like a container, at system start it's a best practice to take advantage of systemd. Podman is capable of helping with this process, lets see how.

We will start deploying a container with Podman the same way we've been doing it during the rest of the workshop.

In [11]:
!podman run --rm -d --name my-autostart-container fedora sleep 99999

Error: creating container storage: the container name "my-autostart-container" is already in use by c8861edda9369326caf047f3eaf0bfafe351bb1481c5df587af8f1ede328a50e. You have to remove that container to be able to reuse that name: that name is already in use, or use --replace to instruct Podman to do so.


Check the container is properly deployed

In [12]:
!podman ps -a

CONTAINER ID  IMAGE                                     COMMAND      CREATED        STATUS         PORTS       NAMES
c8861edda936  registry.fedoraproject.org/fedora:latest  sleep 99999  9 seconds ago  Up 10 seconds              my-autostart-container


Our container is running and it's seen by the operating system as a process. Now we would need to generate a systemd unit file that systemd can use to execute this process when the system starts. Podman provides a very easy way of generating this file from a running container:

In [13]:
!podman generate systemd --new --files --name my-autostart-container


DEPRECATED command:
It is recommended to use Quadlets for running containers and pods under systemd.

Please refer to podman-systemd.unit(5) for details.
/home/ppreciad/Github/wod-podman201/container-my-autostart-container.service


As you can see the output explains that this is a deprecated command, we will come back to that in a minute. First lets take a look at the output file that was generated.

In [14]:
!cat container-my-autostart-container.service

# container-my-autostart-container.service
# autogenerated by Podman 4.8.1
# Thu Dec 21 15:44:15 CET 2023

[Unit]
Description=Podman container-my-autostart-container.service
Documentation=man:podman-generate-systemd(1)
Wants=network-online.target
After=network-online.target
RequiresMountsFor=%t/containers

[Service]
Environment=PODMAN_SYSTEMD_UNIT=%n
Restart=on-failure
TimeoutStopSec=70
ExecStart=/usr/bin/podman run \
	--cidfile=%t/%n.ctr-id \
	--cgroups=no-conmon \
	--rm \
	--sdnotify=conmon \
	--replace \
	-d \
	--name my-autostart-container fedora sleep 99999
ExecStop=/usr/bin/podman stop \
	--ignore -t 10 \
	--cidfile=%t/%n.ctr-id
ExecStopPost=/usr/bin/podman rm \
	-f \
	--ignore -t 10 \
	--cidfile=%t/%n.ctr-id
Type=notify
NotifyAccess=all

[Install]
WantedBy=default.target


As you can see Podman has generated the file for us, the only additional thing we would need to do now is to place this file in ls "$HOME/.config/systemd/user" and reload systemd running "systemctl --user daemon-reload". Once you've done that your system will automatically start your container on boot.

This is cool but, as you've seen, the generated file is long and complex to read specially for non experienced users. Also, most of the times we work with containers we want to work in a declarative manner. This means we would like to have a file in which we define how our application should run and an orchestrator will make it work, this is the default way of working with kubernetes and has become a de-facto standard for containerized environments.

For these reasons Podman introduced Quadlets in version 4.4. This technology brings the capability of writting a simple and easy to read file that can be used how you want you container to run (following the declarative model). Lets see how it works.

The only thing you need to do to take advantage of Quadlets is creating a symple file with the definition of your container and with the name ending in ".container":

In [16]:
cat << EOF > my-quadlet.container
[Unit]
Description=The sleep container
After=local-fs.target

[Container]
Image=docker.io/library/fedora:latest
Exec=sleep 1000

[Install]
# Start by default on boot
WantedBy=multi-user.target default.target
EOF
cat my-quadlet.container


podman version 4.8.1


As you can see this is a very easy to read file that allows you to take advantage of the declarative way of working. Once you have created this file you just need to move it to $HOME/.config/containers/systemd/ if you're a rootless user or to /usr/share/containers/systemd/ if you have root access to the system. Once you've done that reload systemd by running "systemctl --user daemon-reload". Let's do it.

In [23]:
#mkdir ~/.config/containers/systemd/ -p
!mv ./my-quadlet.container ~/.config/containers/systemd/
!systemctl --user daemon-reload

mv: cannot stat './my-quadlet.container': No such file or directory


Now we can use standard linux commands to check the status of our workload:

In [25]:
!systemctl --user status my-quadlet.container

Unit my-quadlet.container.service could not be found.


But that's not all!!! The best part of Quadlets is that it can take kubernetes yaml files as unit files, making it super easy to transition from Podman to kubernetes and the other way around. If you know the basics of kubernetes you may be thinking that isn't possible as kubernetes works with pods as the minimum schedulable unit while Podman works with containers, but Podman can also work with pods. We will explain how to manage pods with Podman in next section.